# Korean Embedding 모델을 SageMaker 배포 및 추론
- 이 노트북은 SageMaker Notebook Instance 의 conday_pytorch_p39 에서 테스트 되었습니다.

Model Ref:
- BM-K/KoSimCSE-roberta
    - https://huggingface.co/BM-K/KoSimCSE-roberta
Inference Code Ref:    
- Huggingface Sagemaker-sdk - Deploy 🤗 Transformers for inference
    - https://github.com/huggingface/notebooks/blob/main/sagemaker/11_deploy_model_from_hf_hub/deploy_transformer_model_from_hf_hub.ipynb
- Sentence Embeddings with Hugging Face Transformers, Sentence Transformers and Amazon SageMaker - Custom Inference for creating document embeddings with Hugging Face's Transformers
    - https://github.com/huggingface/notebooks/blob/main/sagemaker/17_custom_inference_script/sagemaker-notebook.ipynb
    

# 1. HF Hub로 부터 모델 및 토큰 나이저 로딩

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer

def cal_score(a, b):
    '''
    코사인 유사도 구하는 함수
    '''
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta')



## 추론 테스트
- 아래 첫문장, 두번째 문장의 유사도를 구함
- 아래 첫문장, 세째 문장의 유사도를 구함
- 최종적으로 유사도 수치를 비교 함

In [2]:
sentences1 = ['이번 주 일요일에 분당 이마트 점은 문을 여나요',
             '일요일에 분당 이마트는 문 열어요?',
             '분당 이마트 점은 토요일에 몇 시까지 하나요']


def show_embedding_score(tokenizer, model, sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    embeddings, _ = model(**inputs, return_dict=False)

    score01 = cal_score(embeddings[0][0], embeddings[1][0])
    score02 = cal_score(embeddings[0][0], embeddings[2][0])

    print(score01, score02)
    
    
show_embedding_score(tokenizer, model, sentences1)    

tensor([[92.7287]], grad_fn=<MulBackward0>) tensor([[79.8030]], grad_fn=<MulBackward0>)


In [3]:
sentences2 = ['분당 이마트점에 KT 대리점이 있나요?',
             '거기 이마트점에 KT 대리점이 있나요?',
             '분당 아미트 점은 지하 주차장이 있나요?']

show_embedding_score(tokenizer, model, sentences2)


tensor([[89.2612]], grad_fn=<MulBackward0>) tensor([[53.1729]], grad_fn=<MulBackward0>)


# 2. 세이지 메이커로 모델 배포

In [4]:
import sagemaker
import boto3

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

print(f"sagemaker role arn: {role}")

sagemaker role arn: arn:aws:iam::057716757052:role/mlops-blog-ncf-gsmoon


## HF Model ID, HF_TASK 정의

In [5]:
from sagemaker.huggingface import HuggingFaceModel

# Hub Model configuration. https://huggingface.co/models
hub = {
  'HF_MODEL_ID':'BM-K/KoSimCSE-roberta', # model_id from hf.co/models
  'HF_TASK':'feature-extraction'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.26", # transformers version used
   pytorch_version="1.13", # pytorch version used
   py_version="py39", # python version of the DLC
)

## 모델 배포

In [6]:
%%time

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

-----!CPU times: user 241 ms, sys: 9.01 ms, total: 250 ms
Wall time: 3min 2s


# 3. 추론

In [7]:
import numpy as np

payload_1 = {
    "inputs" : sentences1
}

payload_2 = {
    "inputs" : sentences2
}



def predict_payload(data):
    res = predictor.predict(data=data)
    res = np.array(res) # .squeeze().squeeze()
    # print("res: ", res.shape)
    # print("embedding dimension: ", len(res[0][0][0]))
    return res



In [12]:
def show_embedding_score2(payload):
    '''
    # res 
    # 1st dim: samples, 2nd dim: place_hoder, 3rd_dim : CLS, ohter tokens 
    # res.shape --> (3,1)
    # len(res[1][0]) --> 11 두번째 샘플의 11개 토큰
    # len(res[1][0][0]) --> 두번째 샘플의 , 첫번째 토큰 임베딩 (764 사이즈)
    '''
    res = predict_payload(payload)    
    embeddings_0 = torch.Tensor(res[0][0][0]) 
    embeddings_1 = torch.Tensor(res[1][0][0])
    embeddings_2 = torch.Tensor(res[2][0][0])

    score01 = cal_score(embeddings_0, embeddings_1)
    score02 = cal_score(embeddings_0, embeddings_2)    
    print(score01, score02)
    


In [13]:
print("payload_1: \n", payload_1)
show_embedding_score2(payload_1)

payload_1: 
 {'inputs': ['이번 주 일요일에 분당 이마트 점은 문을 여나요', '일요일에 분당 이마트는 문 열어요?', '분당 이마트 점은 토요일에 몇 시까지 하나요']}
tensor([[92.7287]]) tensor([[79.8030]])


/tmp/ipykernel_96458/1901276280.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res = np.array(res) # .squeeze().squeeze()


In [14]:
print("payload_2: \n", payload_2)
show_embedding_score2(payload_2)

payload_2: 
 {'inputs': ['분당 이마트점에 KT 대리점이 있나요?', '거기 이마트점에 KT 대리점이 있나요?', '분당 아미트 점은 지하 주차장이 있나요?']}
tensor([[89.2611]]) tensor([[53.1729]])


/tmp/ipykernel_96458/1901276280.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res = np.array(res) # .squeeze().squeeze()


# 4. 엔드포인트 삭제

In [15]:
# delete endpoint
predictor.delete_model()
predictor.delete_endpoint()